## Things to represent
- The actual utility functions
- 

In [1]:
import ast

import pycid
import numpy as np
from pycid.core.mechanised_graph import MechanisedGraph
from pycid.core.cpd import StochasticFunctionCPD


c:\Users\jhr\Anaconda3\envs\pycid\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def utility_function_follower(defender: tuple, attacker: int) -> float:
    """From GT slides"""
    attacker_reward = 2
    follower_penalty = 1
    u_follower = (1 - defender[attacker-1]) * attacker_reward + defender[attacker-1] * follower_penalty
    return u_follower

def utility_function_leader(d1: tuple, d2: int) -> float:
    leader_reward = 2
    leader_penalty = 1
    u_leader = (1 - d1[d2-1]) * leader_penalty + d1[d2-1] * leader_reward
    return u_leader

def stochastic_cpd_to_dict(stochastic_cpd: StochasticFunctionCPD) -> dict:
    """Converts a stochastic cpd to a dictionary"""
    return ast.literal_eval(str(stochastic_cpd).split("->")[1])


def calculate_utilities(stochastic_cpd: list[StochasticFunctionCPD]) -> tuple[float, float]:
    """Calculates the utility of a stochastic cpd"""
    leader_strategy = stochastic_cpd_to_dict(stochastic_cpd[0])
    follower_strategy = stochastic_cpd_to_dict(stochastic_cpd[1])

    # find keys with nonzero values
    leader_strategy = {key: value for key, value in leader_strategy.items() if value > 0}

    # calculate weighted utility for each
    leader_utility = 0
    follower_utility = 0 
    for leader_strat, leader_prob in leader_strategy.items():
        for follower_strat, follower_prob in follower_strategy.items():
            leader_utility += leader_prob * follower_prob * utility_function_leader(leader_strat, follower_strat)
            follower_utility += leader_prob * follower_prob * utility_function_follower(leader_strat, follower_strat)
    
    return leader_utility, follower_utility

def find_best_nash_equilibrium(nash_eq: list[StochasticFunctionCPD]) -> tuple[StochasticFunctionCPD, float]:
    """Finds the best nash equilibrium"""
    best_nash_eq = None
    best_utility = 0
    for eq in nash_eq:
        leader_utility, follower_utility = calculate_utilities(eq)
        if leader_utility > best_utility:
            best_utility = leader_utility
            assert isinstance(best_utility, float)
            best_nash_eq = eq
    return best_nash_eq, best_utility


In [13]:


# Simple MACID setup
macid = pycid.MACID(
    [("D1", "U1"), ("D1", "U2"), ("D2", "U1"), ("D2", "U2")],
    agent_decisions={1: ["D1"], 2: ["D2"]},
    agent_utilities={1: ["U1"], 2: ["U2"]},
)

# Discretise domains to fit with PyCID (It is not possible to use continuous domains in pygambit, right?)
PRECISION = 0.2
CONSTRAINT = 2.6
d1_domain = [(a, b, c) for a in np.arange(0, 1.1, PRECISION) for b in np.arange(0, 1.1, PRECISION) for c in np.arange(0, 1.1, PRECISION) if a + b + c == CONSTRAINT]
d2_domain = list(range(1, 4))

macid.add_cpds(
    D1 = d1_domain,
    D2 = d2_domain,
    U1 = lambda D1, D2: utility_function_leader(D1, D2),
    U2 = lambda D1, D2: utility_function_follower(D1, D2),
)


mech = MechanisedGraph(macid)

In [14]:
# solve for all nash equilibria
nash_eq = macid.get_ne(solver="enummixed")

C:\Users\jhr\pycid\pycid\core\macid_base.py:59: UserWarning: adding DecisionDomain to non-decision node D1
  warn(f"adding DecisionDomain to non-decision node {variable}")
C:\Users\jhr\pycid\pycid\core\macid_base.py:59: UserWarning: adding DecisionDomain to non-decision node D2
  warn(f"adding DecisionDomain to non-decision node {variable}")


In [20]:
from pycid.core.cpd import StochasticFunctionCPD

def any_high_prob(CPD: StochasticFunctionCPD, threshold=0.9) -> bool:
    return any(probability for probability in CPD.get_values() if probability > threshold)

In [25]:
[calculate_utilities(eq) for eq in nash_eq]

[(1.8666666666666671, 1.1333333333333333),
 (1.8666666666666667, 1.1333333333333333),
 (1.8666666666666667, 1.1333333333333333),
 (1.8666666666666667, 1.1333333333333333),
 (1.866666666666667, 1.1333333333333335)]

In [23]:
import random
#sample_nash_eqs = random.sample(nash_eq, 200)

best_equilibrium, best_utility = find_best_nash_equilibrium(nash_eq)
print(f"{best_utility} for the leader")

1.8666666666666671 for the leader
